# Helper methods, imports, setup

In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import smtplib
import random
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

app = Flask(__name__)

THRESHOLDS_SS = {
    "Methadon": 2,
    "Dormicum": 1,
    "Bupaq": 3,
    "Esconarkon": 3,
    "Keta-S": 4,
    "Propofol": 2
}

THRESHOLDS_BS = {
    "Methadon": 1000,
    "Dormicum": 500,
    "Bupaq": 1500,
    "Esconarkon": 1500,
    "Keta-S": 2000,
    "Propofol": 1000
}

def load_inventory(file_path):
    #load inventory from csv
    try:
        return pd.read_csv(file_path, sep=',')
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{file_path}' does not exist.")
    except Exception as e:
        raise Exception(f"An error occurred while loading the inventory: {e}")

# Save inventory
def save_inventory(file_path, inventory):
    #save inventory to csv
    try:
        inventory.to_csv(file_path, index=False)
    except Exception as e:
        raise Exception(f"An error occurred while saving the inventory: {e}")

def update_inventory(file_path, Name, operation, big_safe, amount=None):
    #update inventory for specific product
   
    inventory = load_inventory(file_path)

    thresholds = THRESHOLDS_BS if big_safe else THRESHOLDS_SS
  
    if Name not in inventory['Name'].values:
        raise KeyError(f"Product Name '{Name}' not found in the inventory.")

    #threshold defined?
    if Name not in thresholds and amount is None:
        raise ValueError(f"Threshold not defined for product '{Name}', and no amount provided.")

    #update either using provided amount or double threshold
    if amount is None:
        amount = 2 * thresholds[Name]

    if operation == 'add':
        inventory.loc[inventory['Name'] == Name, 'quantity'] += amount
    elif operation == 'remove':
        inventory.loc[inventory['Name'] == Name, 'quantity'] -= amount
        # Ensure quantity doesn't drop below zero
        inventory.loc[inventory['Name'] == Name, 'quantity'] = \
            inventory.loc[inventory['Name'] == Name, 'quantity'].clip(lower=0)
    else:
        raise ValueError(f"Invalid operation '{operation}'. Supported operations: 'add', 'remove'.")

    save_inventory(file_path, inventory)
    return inventory

def get_thresholds(file_path):
    #get threshholds of small or big inventory
    if "narcotics_ss.csv" in file_path:
        return THRESHOLDS_SS
    elif "narcotics_bs.csv" in file_path:
        return THRESHOLDS_BS
    else:
        raise ValueError("Unsupported file name. Use 'narcotics_ss.csv' or 'narcotics_bs.csv'.")

# API endpoint for retrieving inventory

In [2]:
@app.route('/get_inventory', methods=['GET'])
def get_inventory():
    try:
        file_path = request.args.get('file_path')
        
        if not file_path:
            return jsonify({'error': 'Missing file_path parameter'}), 400
                
        inventory = load_inventory(file_path)
        return jsonify(inventory.to_dict(orient='records')), 200
    except FileNotFoundError as e:
        return jsonify({'error': str(e)}), 404
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# API endpoint for updating inventory

In [3]:
@app.route('/update_inventory', methods=['POST'])
def api_update_inventory():
    try:
        data = request.json
        file_path = data.get('file_path')
        Name = data.get('Name')
        operation = data.get('operation')
        big_safe = data.get('big_safe', False)  # Default False
        amount = data.get('amount')
        

        # Validate input
        if not file_path or not Name or not operation:
            return jsonify({'error': 'Missing required fields: file_path, Name, or operation'}), 400

        updated_inventory = update_inventory(file_path, Name, operation, big_safe, amount)
        return jsonify(updated_inventory.to_dict(orient='records')), 200
    except KeyError as e:
        return jsonify({'error': f'Key error: {str(e)}'}), 400
    except ValueError as e:
        return jsonify({'error': f'Value error: {str(e)}'}), 400
    except Exception as e:
        return jsonify({'error': f'Unexpected error: {str(e)}'}), 500

# API endpoint to determine what has to be reorderd or refilled

In [4]:
@app.route('/evaluate-medications', methods=['POST'])
def evaluate_medications():
    #determine reordering/refilling based on threshold
    try:
        data = request.get_json()

        if not data or 'file_path' not in data:
            return jsonify({'error': 'Missing file_path parameter'}), 400

        file_path = data['file_path']
        inventory = load_inventory(file_path)

        # Ensure inventory has the required columns
        if 'Name' not in inventory.columns or 'quantity' not in inventory.columns:
            return jsonify({'error': 'Invalid inventory format. Missing Name or quantity column.'}), 400

        # Get thresholds based on the file name
        thresholds = get_thresholds(file_path)

        # Filter medications below threshold
        below_threshold = inventory[
            inventory.apply(lambda row: row['Name'] in thresholds and row['quantity'] < thresholds[row['Name']], axis=1)
        ]['Name'].tolist()

        # Create the response in the desired format
        response = {
            "orderItem": [{"narcotic": med} for med in below_threshold]
        }

        return jsonify(response), 200
    except ValueError as e:


        return jsonify({'error': str(e)}), 400
    except Exception as e:
        print(str(e))
        return jsonify({'error': str(e)}), 500

# API Endpoint for simulation of drug shortages

In [5]:
@app.route('/shortage', methods=['POST'])
def check_shortage():
    try:
        # Assuming the input is a JSON object with a single key 'narcotic'
        print(request)
        narcotic_input = request.get_json()
        print(narcotic_input)
        
        # Extracting the narcotic name from the input dictionary
        narcotic = narcotic_input.get('narcotic', '')
        print(narcotic)
        
        if not narcotic:
            return {"error": "No narcotic name provided"}, 400
        
        # List of valid narcotics
        valid_narcotics = ['Methadon', 'Dormicum', 'Bupaq', 'Esconarkon', 'Keta-S', 'Propofol']
        
        if not any(narc.lower() == narcotic.lower() for narc in valid_narcotics):
            return {
                "error": "Invalid narcotic name",
                "valid_narcotics": valid_narcotics
            }, 400
            
        # should work for lower and upper
        narcotic = next(narc for narc in valid_narcotics if narc.lower() == narcotic.lower())
            
        # Randomly determine if there's a shortage (20% chance)
        has_shortage = random.choices([True, False], weights=[20, 80])[0]
        response = {
            "narcotic": narcotic,
            "shortage": has_shortage,
            "message": f"There {'is' if has_shortage else 'is not'} currently a shortage of {narcotic}"
        }
        print(response)
        return response, 200
        
    except Exception as e:
        print(e)
        return {
            "error": "An unexpected error occurred",
            "details": str(e)
        }, 500

# Function and Endpoint for sending emails

In [6]:
# Configuration
SMTP_SERVER = "smtp.gmail.com"
PORT = 587
USERNAME = "vectorgroup209@gmail.com"
PASSWORD = "lydx inzx skaq qqgi"

def send_email(to_email, subject, body, from_email=None):
    try:
        msg = MIMEMultipart()
        msg['From'] = from_email if from_email else USERNAME
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(SMTP_SERVER, PORT) as server:
            server.starttls()
            server.login(USERNAME, PASSWORD)
            server.sendmail(USERNAME, to_email, msg.as_string())

        return {"status": "success", "message": "Email sent successfully"}
    except Exception as e:
        print(str(e))
        return {"status": "error", "message": str(e)}

@app.route('/send-email', methods=['POST'])
def send_email_api():
    data = request.json
    to_email = data.get('to_email')
    subject = data.get('subject')
    body = data.get('body')
    from_email = data.get('from_email', USERNAME)

    if not to_email or not subject or not body:
        return jsonify({"status": "error", "message": "Missing required fields"}), 400

    result = send_email(to_email, subject, body, from_email)
    return jsonify(result)

# Check refill availability for the small safe

In [7]:
@app.route('/check_refill_availability', methods=['POST'])
def check_refill_availability():
    """
    Checks if the doubled small safe thresholds are available in the big safe for specified narcotics.
    """
    try:
        data = request.json
        if not data or 'orderItem' not in data or 'big_safe_file' not in data:
            return jsonify({'error': 'Missing or invalid input data'}), 400

        order_items = data.get('orderItem', [])
        big_safe_file = data.get('big_safe_file')

        if not isinstance(order_items, list):
            return jsonify({'error': '"orderItem" must be a list'}), 400
        if not big_safe_file:
            return jsonify({'error': 'Missing "big_safe_file" in input data'}), 400

        # Load the big safe inventory
        big_safe_inventory = load_inventory(big_safe_file)

        # Ensure the inventory has the required columns
        if 'Name' not in big_safe_inventory.columns or 'quantity' not in big_safe_inventory.columns:
            return jsonify({'error': 'Invalid big safe inventory format. Missing Name or quantity column.'}), 400

        shortages = []

        # Process each narcotic in the order
        for item in order_items:
            narcotic = item.get('narcotic')
            if not narcotic:
                continue

            # Check if the narcotic is in the small safe thresholds
            if narcotic not in THRESHOLDS_SS:
                shortages.append(f'Narcotic "{narcotic}" is not defined in small safe thresholds.')
                continue

            # Check availability for the narcotic
            threshold = THRESHOLDS_SS[narcotic]
            required_amount = 2 * threshold
            big_safe_quantity = big_safe_inventory.loc[
                big_safe_inventory['Name'] == narcotic, 'quantity'
            ].sum()  # Sum in case there are duplicates

            big_safe_quantity = int(big_safe_quantity)

            if big_safe_quantity < required_amount:
                shortages.append(
                    f"{narcotic}: Insufficient stock in big safe. "
                    f"Required: {required_amount}, Available: {big_safe_quantity}."
                )

        # Return shortages or an empty string if no shortages exist
        if shortages:
            print("Shortages Found:", shortages)
            return jsonify({'shortage_message': '\n'.join(shortages)}), 200
        print("No Shortages")
        return jsonify({'shortage_message': ''}), 200

    except FileNotFoundError as e:
        print("FileNotFoundError:", str(e))
        return jsonify({'error': str(e)}), 404
    except Exception as e:
        print("Unexpected Exception:", str(e))
        return jsonify({'error': str(e)}), 500


# Run API

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.3.57.22:8080
Press CTRL+C to quit
172.3.28.49 - - [15/Dec/2024 15:22:34] "POST /evaluate-medications HTTP/1.1" 200 -
172.3.186.144 - - [15/Dec/2024 15:22:34] "POST /evaluate-medications HTTP/1.1" 200 -
172.3.50.42 - - [15/Dec/2024 15:22:35] "POST /check_refill_availability HTTP/1.1" 400 -
Incoming JSON Data: {'orderItem': {'orderItem': [{'narcotic': 'Methadon'}]}, 'big_safe_file': '/work/narcotics_bs.csv'}
172.3.186.144 - - [15/Dec/2024 15:23:15] "POST /check_refill_availability HTTP/1.1" 200 -
Incoming JSON Data: {'orderItem': [{'narcotic': 'Methadon'}], 'big_safe_file': '/work/narcotics_bs.csv'}
No Shortages
172.3.50.42 - - [15/Dec/2024 15:26:08] "POST /evaluate-medications HTTP/1.1" 200 -
172.3.50.42 - - [15/Dec/2024 15:26:08] "POST /evaluate-medications HTTP/1.1" 200 -
172.3.28.49 - - [15/Dec/2024 15:26:08] "POST /check_refill_availab

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=945152e0-66a8-4b9c-b068-364219c8e551' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>